In [1]:
import pandas as pd
import cPickle as pickle
import sqlite3
import numpy as np
from gensim.models.doc2vec import LabeledSentence
import cjson

In [4]:
df = pd.DataFrame(np.arange(9).reshape(3,3))

In [5]:
print df.size
print df.shape

9
(3, 3)


In [2]:
pathway = '../../data/labeledRedditComments2.p'
path2 = '../../data/RedditMay2015Comments.sqlite'

In [3]:
df = pickle.load(open(pathway, 'rb'))

In [4]:
def ishateful(subreddit):
    '''
    Input: str subreddit
    Output: int 1 if hateful subreddit, 0 otherwise
    '''

    # List of not hateful subreddits
    final_nothate_srs = ['politics', 'worldnews', 'history', 'blackladies', 'lgbt',
                         'TransSpace', 'women', 'TwoXChromosomes', 'DebateReligion',
                         'religion', 'islam', 'Judaism', 'BodyAcceptance', 'fatlogic'
                         'gaybros','AskMen','AskWomen']
    # List of hateful subreddits
    final_hateful_srs = ['CoonTown', 'WhiteRights', 'Trans_fags', 'SlutJustice',
                         'TheRedPill', 'KotakuInAction', 'IslamUnveiled', 'GasTheKikes',
                         'AntiPOZi', 'fatpeoplehate', 'TalesofFatHate','hamplanethatred',
                         'shitniggerssay','neofag','altright']

    if subreddit in final_hateful_srs:
        return 1
    else:
        return 0

In [5]:
df.columns

Index([u'subreddit', u'id', u'name', u'body', u'label'], dtype='object')

In [6]:
df.label.value_counts()

0    1234794
1     565494
Name: label, dtype: int64

In [4]:
df.columns

Index([u'subreddit', u'id', u'name', u'body', u'label'], dtype='object')

In [6]:
numcomments = len(df.index)

In [10]:
df['subreddit'].value_counts()

worldnews          490354
fatpeoplehate      311183
politics           244927
AskMen             138839
AskWomen           137889
KotakuInAction     128156
TwoXChromosomes    105130
TheRedPill          59145
CoonTown            51979
DebateReligion      41015
islam               25443
history             25242
Judaism              9103
lgbt                 8253
TalesofFatHate       5239
AntiPOZi             4740
blackladies          4396
religion             2623
Trans_fags           2362
WhiteRights          1352
GasTheKikes           919
BodyAcceptance        579
women                 529
TransSpace            472
SlutJustice           309
IslamUnveiled         110
Name: subreddit, dtype: int64

In [11]:
df['label'].value_counts()

NotHate    1234794
Hate        565494
Name: label, dtype: int64

In [12]:
565494.0/1234794.0

0.45796626805766794

In [35]:
def reddit_comment_gen(df):
''' Input: a pandas df
    Output: this is a generator that gives the next row in the df'''  

    numrows = df.size
    for row in xrange(numrows):


        # load a comment
        comment = df.iloc[row,:]

        # Load the JSON object
        body = str(comment['body'])
        subreddit = str(comment['subreddit'])

        # Clean and tokenize text
#             body = text_cleaner(body)

        # generate
#             yield LabeledSentence(body,labels=[str(json_object['subreddit'])])
        yield "{}: {}".format(subreddit,body)

In [36]:
mygen = reddit_comment_gen(df)

In [41]:
mygen.next()

'CoonTown: Someone has deeper internal issues they have with themselves. '

In [46]:
conn = sqlite3.connect(path2)
c = conn.cursor()
c.execute('SELECT subreddit, body FROM MAY2015 LIMIT 5')

In [49]:
val = c.next()

In [53]:
val[1]

u"gg this one's over. off to watch the NFL draft I guess"

In [54]:
type(c)

sqlite3.Cursor

In [56]:
for i in c:
    print i

(u'politics', u"Are you really implying we return to those times or anywhere near that political environment?  If so, you won't have much luck selling the American people on that governance concept without ushering in American Revolution 2.0.")
(u'AskReddit', u"No one has a European accent either  because it doesn't exist. There are accents from Europe but not a European accent.")
(u'AskReddit', u'That the kid "..reminds me of Kevin."   so sad :-(')


In [84]:
def sql_gen(c):
''' Input: sqlite3 cursor to a sqlite3 database
    Output: this is a generator that gives the next query result from c'''  

    for comment in c:
        try:
            subreddit = str(comment[0])
            body = str(comment[1])
            yield "{}: {}".format(subreddit,body)
        except:
            yield None

# need except statement because sometimes the body or subreddit can't be converted to a str'
#         yield comment

In [90]:
conn = sqlite3.connect(path2)
c = conn.cursor()
c.execute('SELECT subreddit, body FROM MAY2015')
mygen = sql_gen(c)

Exception RuntimeError: 'generator ignored GeneratorExit' in <generator object sql_gen at 0x7f514691adc0> ignored


In [93]:
print mygen.next()

politics: Are you really implying we return to those times or anywhere near that political environment?  If so, you won't have much luck selling the American people on that governance concept without ushering in American Revolution 2.0.


In [95]:
for i in xrange(100):
    print mygen.next()
    print ""

gaming: Sweet , great Idea there as well 

BabyBumps: Thanks for the heads up! Good to know!

trees: Bruh

IAmA: VERY looking forward to *Pollo Empire* once Better Call Saul finishes it's sixth season!

nfl: Can we sticky these main 4 threads? 

WTF: what does it look like now?

Frugal: Right, the tens of thousands of people who installed their own all killed themselves.

Electricity is only dangerous if you're stupid and don't follow the electrical code.

Its very simple stuff that you can pick up if you're handy and can read.

Do have a licensed electrician install the crossover switch as most municipalities require that.


CoDCompetitive: Some hands don't physically allow you to claw 

AskReddit: Go back to Route 21, dude.

WTF: I'm trying to eat a fucking fajita here dammit

lgbt: Thank you for your advice. I'll try to remember that this all takes time-even though it's frustrating :)

trees: Legalize medical marijuana in 2001 too! Just goes to show how forward thinking this guy is.